In [2]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from colorama import Fore
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from tld import get_tld, is_tld
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier
#import numpy as np
#from flask import Flask, request, url_for, render_template


def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0
    
def httpSecure(url):
    htp = urlparse(url).scheme
    match = str(htp)
    if match=='https':
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0  

def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits 


def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters
    
  
def Shortining_Service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0
    
    

def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4 with port
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}|'
        '([0-9]+(?:\.[0-9]+){3}:[0-9]+)|'
        '((?:(?:\d|[01]?\d\d|2[0-4]\d|25[0-5])\.){3}(?:25[0-5]|2[0-4]\d|[01]?\d\d|\d)(?:\/\d{1,2})?)', url)  # Ipv6
    if match:
        return 1
    else:
        return 0 



def ml_system(data):
    df_new = pd.read_csv("comprehensive_phishing_data.csv")
    
    df_new = df_new.sample(frac = 0.05)
    X = df_new.drop(['url', 'target'],axis=1)#,'type_code'
    y = df_new['target']

    ##split dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

    ###Ensemble learning 
    kfold = model_selection.KFold(n_splits=10)
    model1 = DecisionTreeClassifier(random_state =0)
    model2 = RandomForestClassifier(n_estimators=10)
    model3 = KNeighborsClassifier()
    model4 = GaussianNB()
    # create the sub models
    estimators = []
    estimators.append(('Decision_Tree', model1))
    estimators.append(('Random_Forest', model2))
    estimators.append(('KNN', model3))
    estimators.append(('NB', model4))
    # create the ensemble model
    ensemble = VotingClassifier(estimators)
    ##fit data
    ensemble.fit(X_train,y_train)
    arr_data = []
    for n in data.values:
        n_values = n[1:]
        for i in n_values:
            arr_data.append(i)
    pred_value = ensemble.predict([arr_data])
    for k in pred_value:
        if k==0:
            print("Benign")
        else:
            print("Phishing")
            
            


url_link = []
url_val = input("enter url: ")
url_link.append(url_val)
data = pd.DataFrame(url_link, columns=["url"])

data["url"] = data['url'].replace('www.', '', regex=True)
data['url_len'] = data['url'].apply(lambda x: len(str(x)))
feature = ['@','?','-','=','.','#','%','+','$','!','*',',','//']
for a in feature:
    data[a] = data['url'].apply(lambda i: i.count(a))
data['abnormal_url'] = data['url'].apply(lambda i: abnormal_url(i))
data['https'] = data['url'].apply(lambda i: httpSecure(i))
data['digits']= data['url'].apply(lambda i: digit_count(i))
data['letters']= data['url'].apply(lambda i: letter_count(i))
data['Shortining_Service'] = data['url'].apply(lambda x: Shortining_Service(x))
data['having_ip_address'] = data['url'].apply(lambda i: having_ip_address(i))

ml_system(data)

enter url: www.google.com
Benign
